In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

C:\Users\LMS-Sapkals\AppData\Local\Temp\ipykernel_20036\4110068169.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [2]:
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time
import timeit #imports timeit module
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
start_time = timeit.default_timer() #defines start time so computational time can be calculated
print("Hello World")
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

Hello World
---Run time is 0.00012390001211315393 seconds ---



In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import statistics
import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
import wfdb
import numpy as np
import pandas as pd

In [4]:
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir('/Users/chenc/000---STT450-Jupyter Notebook/000--STT450-550/') 
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:15<00:00,  1.58it/s]


In [5]:
# df3 = pd.read_csv('Data/Carseats.csv')
# #df3.info()
# print(); df3.head(5)
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
features_maindf = pd.concat(features_df_list, ignore_index = True)

In [6]:
# df3['High'] = df3.Sales.map(lambda x: 1 if x>8 else 0)
# df3.ShelveLoc = pd.factorize(df3.ShelveLoc)[0]
# df3.Urban = df3.Urban.map({'No':0, 'Yes':1})
# df3.US = df3.US.map({'No':0, 'Yes':1})
# #df3.info()
# print(); df3.head(5)
#X = df3.drop(['Sales', 'High'], axis = 1)
#y = df3.High
##print(X.shape)
#print(y.shape)

In [7]:
x_values = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn']
y_values = ['Rhythm']
#features_df_list[0]

### Decision Tree

criterion{“gini”, “entropy”}, default=”gini”

In [8]:
# #for df in features_df_list:
# #model = DecisionTreeClassifier(max_depth = 6) #criterion='entropy'
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# max_depth_list = []
# n_estimators_list = []
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     temp_model = DecisionTreeClassifier(max_depth = 6)    
#     param_distributions = {'max_depth': randint(7,9),
#                           'nestim'}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])
    
#     model = DecisionTreeClassifier(max_depth = 8) #criterion='entropy'
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 
# elapsed = timeit.default_timer() - start_time #gives total computation time
# print("---Run time is %s seconds ---" % elapsed) #prints computation time
# print()

# print('Accuracy of each fold: \n {}'.format(acc_score))
# print()
# print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
# print()
# print('Std of accuracy : \n{}'.format(np.std(acc_score)))

In [9]:
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(acc_score)
# print(elapsed)
# #print(n_estimators_list)
# # print(max_depth_list)

# # temp_list = []
# # temp_idx = []
# # for acc in acc_score:
# #     if acc >= 0.95:
# #         temp_list.append(acc)
# #         temp_idx.append(acc_score.index(acc))
# # for idx in temp_idx:
# #     print(max_depth_list[idx])
    
# # statistics.mode(max_depth_list)

In [10]:
# [[22543  2200]
#  [ 2067 18158]]
#               precision    recall  f1-score   support

#            0       0.92      0.91      0.91     24743
#            1       0.89      0.90      0.89     20225

#     accuracy                           0.91     44968
#    macro avg       0.90      0.90      0.90     44968
# weighted avg       0.91      0.91      0.91     44968

In [11]:
# [[22796  1947]
#  [ 1763 18462]]
#               precision    recall  f1-score   support

#            0       0.93      0.92      0.92     24743
#            1       0.90      0.91      0.91     20225

#     accuracy                           0.92     44968
#    macro avg       0.92      0.92      0.92     44968
# weighted avg       0.92      0.92      0.92     44968

In [12]:
# Avg accuracy : 
# 0.9225946312949485

# Std of accuracy : 
# 0.10175840774589326

# [[22670  2073]
#  [ 1762 18463]]
#               precision    recall  f1-score   support

#            0       0.93      0.92      0.92     24743
#            1       0.90      0.91      0.91     20225

#     accuracy                           0.91     44968
#    macro avg       0.91      0.91      0.91     44968
# weighted avg       0.91      0.91      0.91     44968

# Bagging: use p for classification

In [13]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
max_depth_list = []
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]
    
    temp_model = RandomForestClassifier(max_features=16,random_state = 2)    
    param_distributions = {'max_depth': randint(11,14),
                          'n_estimators': randint(115,139)}

    search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
    search.fit(X_train, y_train)

    n_estimators_list.append(search.best_params_['n_estimators'])
    max_depth_list.append(search.best_params_['max_depth'])
    
    model = RandomForestClassifier(max_features = 16, random_state = 2, max_depth=search.best_params_['max_depth'], n_estimators=search.best_params_['n_estimators'])
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(acc_score)
print(elapsed)
print(n_estimators_list)
print(max_depth_list)
temp_list = []
temp_idx = []
n_estimators_best = []
max_depth_best = []
for acc in acc_score:
    if acc >= 0.95:
        temp_list.append(acc)
        temp_idx.append(acc_score.index(acc))
for idx in temp_idx:
    n_estimators_best.append(n_estimators_list[idx])
    max_depth_best.append(max_depth_list[idx])

23it [4:14:35, 664.13s/it]


---Run time is 15291.779910900004 seconds ---

Accuracy of each fold: 
 [0.9130434782608695, 0.9505070993914807, 0.9955919395465995, 0.9244604316546763, 0.9973876698014629, 0.954158215010142, 0.7986921999065857, 0.9979577944179714, 0.9417670682730924, 0.9680264608599779, 0.9813974591651543, 0.9885139985642498, 0.8326530612244898, 0.9471337579617835, 0.6144879267277269, 0.9871510855117412, 0.9869773817683345, 0.9873052510098096, 0.7303465765004227, 0.9708631115997801, 0.9931565440547476, 0.997489014438167, 0.9953488372093023]

Avg accuracy : 
0.9328007114286333

Std of accuracy : 
0.09622327149837745
[[23035  1708]
 [ 1652 18573]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     24743
           1       0.92      0.92      0.92     20225

    accuracy                           0.93     44968
   macro avg       0.92      0.92      0.92     44968
weighted avg       0.93      0.93      0.93     44968

[0.9130434782608695, 0.950507099391

TypeError: 'list' object is not callable

In [10]:
statistics.mode([13, 11, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 13, 12, 13, 13, 11, 11, 13, 11, 13, 11])

13

In [14]:
for acc in acc_score:
    if acc >= 0.95:
        temp_list.append(acc)
        temp_idx.append(acc_score.index(acc))
for idx in temp_idx:
    n_estimators_best.append(n_estimators_list[idx])
    max_depth_best.append(max_depth_list[idx])

In [12]:

#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
# n_estimators_list = []
# max_depth_list = []
import statistics
#mode_depth = statistics.mode(max_depth_best)
#mode_estimators = statistics.mode(n_estimators_best)
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]
    
#     temp_model = RandomForestClassifier(max_features=16,random_state = 2)    
#     param_distributions = {'max_depth': randint(11,14),
#                           'n_estimators': randint(115,139)}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])
    
    model = RandomForestClassifier(max_features = 16, random_state = 2, max_depth=13, 
                                   n_estimators=127)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(acc_score)
print(elapsed)
print(n_estimators_list)
print(max_depth_list)
# temp_list = []
# temp_idx = []
# for acc in acc_score:
#     if acc >= 0.95:
#         temp_list.append(acc)
#         temp_idx.append(acc_score.index(acc))
# for idx in temp_idx:
#     print(n_estimators_list[idx])
#     print(max_depth_list[idx])
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(acc_score)
# print(elapsed)
# print(n_estimators_list)
# print(max_depth_list)

# temp_list = []
# temp_idx = []
# for acc in acc_score:
#     if acc >= 0.95:
#         temp_list.append(acc)
#         temp_idx.append(acc_score.index(acc))
# for idx in temp_idx:
#     print(n_estimators_list[idx])
#     print(max_depth_list[idx])

23it [06:26, 16.80s/it]

---Run time is 693.3505415000109 seconds ---

Accuracy of each fold: 
 [0.9130434782608695, 0.9496957403651115, 0.9955919395465995, 0.9238609112709832, 0.9973876698014629, 0.9545638945233266, 0.7986921999065857, 0.9979577944179714, 0.9422690763052208, 0.9685777287761852, 0.9813974591651543, 0.9885139985642498, 0.8331065759637188, 0.9503184713375796, 0.6194837635303914, 0.9871510855117412, 0.9869773817683345, 0.9873052510098096, 0.7417582417582418, 0.9708631115997801, 0.9918733960650128, 0.997489014438167, 0.9961945031712474]

Avg accuracy : 
0.9336553342199018

Std of accuracy : 
0.09447902543178308
[[23064  1679]
 [ 1637 18588]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     24743
           1       0.92      0.92      0.92     20225

    accuracy                           0.93     44968
   macro avg       0.93      0.93      0.93     44968
weighted avg       0.93      0.93      0.93     44968

[0.9130434782608695, 0.949695740365

NameError: name 'n_estimators_list' is not defined

In [13]:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
# n_estimators_list = []
# max_depth_list = []
import statistics
#mode_depth = statistics.mode(max_depth_best)
#mode_estimators = statistics.mode(n_estimators_best)
#confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]
    
#     temp_model = RandomForestClassifier(max_features=16,random_state = 2)    
#     param_distributions = {'max_depth': randint(11,14),
#                           'n_estimators': randint(115,139)}

#     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
#     search.fit(X_train, y_train)

#     n_estimators_list.append(search.best_params_['n_estimators'])
#     max_depth_list.append(search.best_params_['max_depth'])
    
    model = RandomForestClassifier(max_features = 16, random_state = 2, max_depth=5)
    model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list 
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(acc_score)
print(elapsed)

23it [02:25,  6.35s/it]

---Run time is 1262.5597275000036 seconds ---

Accuracy of each fold: 
 [0.9061784897025171, 0.9456389452332657, 0.9955919395465995, 0.908273381294964, 0.9952978056426333, 0.9440162271805274, 0.8397944885567492, 0.9972770592239619, 0.9332329317269076, 0.958103638368247, 0.9818511796733213, 0.9849246231155779, 0.782312925170068, 0.9738853503184713, 0.5578684429641965, 0.973859105006646, 0.9835503769705277, 0.9930755914598961, 0.7071005917159763, 0.9763606377130292, 0.9914456800684346, 0.9899560577526679, 0.9839323467230444]

Avg accuracy : 
0.9262403397881838

Std of accuracy : 
0.10672839222741957
[[22771  1972]
 [ 1738 18487]]
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     24743
           1       0.90      0.91      0.91     20225

    accuracy                           0.92     44968
   macro avg       0.92      0.92      0.92     44968
weighted avg       0.92      0.92      0.92     44968

[0.9061784897025171, 0.94563894523326

In [ ]:
import statistics
statistics.mode(n_estimators_list)
statistics.mode(max_depth_list)

In [ ]:
# [[23029  1714]
#  [ 1649 18576]]
#               precision    recall  f1-score   support

#            0       0.93      0.93      0.93     24743
#            1       0.92      0.92      0.92     20225

#     accuracy                           0.93     44968
#    macro avg       0.92      0.92      0.92     44968
# weighted avg       0.93      0.93      0.93     44968
# ---Run time is 27743.89577069995 seconds ---

# Accuracy of each fold: 
#  [0.914187643020595, 0.9509127789046653, 0.9955919395465995, 0.9250599520383693, 0.9973876698014629, 0.9565922920892495, 0.79542269967305, 0.9979577944179714, 0.9422690763052208, 0.9680264608599779, 0.9813974591651543, 0.9885139985642498, 0.8335600907029479, 0.9496815286624204, 0.6186511240632806, 0.9858218874612317, 0.9876627827278959, 0.9855741488747836, 0.7324598478444633, 0.971412864211105, 0.9940119760479041, 0.997489014438167, 0.9936575052854123]

# Avg accuracy : 
# 0.9331870667263553

# Std of accuracy : 
# 0.09555730626481958

In [ ]:
#help(confusion_matrix)

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

#     model = RandomForestClassifier(max_features = 10, random_state = 2)
#     model.fit(X_train,y_train.values.ravel())

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 


# model = RandomForestClassifier(max_features = 16, random_state = 2)
# scores = cross_val_score(model, X, y, cv=crossvalidation)
# print('Accuracy of each fold: \n {}'.format(scores))
# print()
# print("Avg accuracy: {}".format(scores.mean()))

### An AdaBoost classifier.

The argument `n_estimators = 500` indicates that we want 500 trees

class sklearn.ensemble.AdaBoostClassifier(base_estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
    
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 

In [ ]:
# print(acc_score)
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(confusion_matrix_df)

### Gradient Boosting for classification

In [ ]:
# crossvalidation = KFold(n_splits=23)
# #for df in features_df_list:
# df = features_maindf
# acc_score = [];
# Truth = [];
# Output = [];
# #confusion_matrix_df = pd.DataFrame(columns=['TN', 'FP', 'FN', 'TP'])
# for idx, subject in tqdm(enumerate(good_list)):
    
#     train_index = ~df['Subject'].str.contains(subject)
#     test_index = df['Subject'].str.contains(subject)
#     X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
#     y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

#     model =  GradientBoostingClassifier(n_estimators = 500, 
#                                            learning_rate = 0.1, 
#                                            max_depth = 4, 
#                                            random_state = 2)
#     model.fit(X_train,y_train)

#     pred_values = model.predict(X_test)
#     pred_prob = model.predict_proba(X_test)[:, 1]
     
#     acc = accuracy_score(y_test, pred_values)
#     acc_score.append(acc)
#     #confusion_matrix_df.loc[idx] = confusion_matrix(Truth, Output).ravel()
#     Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
#     Output.extend(pred_values); ## it is a list 

In [ ]:
# print(acc_score)
# print(confusion_matrix(Truth, Output))
# print(classification_report(Truth, Output))
# print(confusion_matrix_df)